In [ ]:
import scDML
print(scDML.__version__)
import scanpy as sc 
import torch
from scDML import scDMLModel
from scDML.utils import print_dataset_information
import matplotlib.pyplot as plt
import pandas as pd
import os
os.system("clear")

In [ ]:
import scanpy as sc 
import pandas as pd


In [ ]:
dataset = 'Lung'

batch_key_dic = {'Immune_Human' : 'batch',
                 'Lung' : 'batch',
                 'Pancreas' : 'tech',
                'Human_Retina': "Batch"}
cell_type_key_dic = {'Immune_Human' : 'final_annotation',
                 'Lung' : 'cell_type',
                 'Pancreas' : 'celltype',
                 "Human_Retina":"Subcluster"}
batch = batch_key_dic[dataset]
cell_type = cell_type_key_dic[dataset]
data_path="/home/krushna/Documents/Data_integration/SCRNA_Datasets/All_h5ad/"

In [ ]:
data_path += dataset+".h5ad"
adata=sc.read(data_path)
adata.obs['BATCH'] = adata.obs[batch]
ncluster = len(adata.obs[cell_type].unique())

In [ ]:
import time
start_time = time.time()

In [ ]:
save_dir="./test_result/"+dataset+"/"

scdml=scDMLModel(save_dir=save_dir)
adata_int=scdml.preprocess(adata,cluster_method="louvain",resolution=3.0,batch_key = 'BATCH')
#print(adata)
scdml.integrate(adata_int,batch_key='BATCH',ncluster_list=[ncluster],
               expect_num_cluster=ncluster,merge_rule="rule2")

# plt.plot(range(1,len(scdml.loss)+1),scdml.loss)

In [ ]:
end_time = time.time()
print('time taken to run :', end_time - start_time)

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

sc.pp.neighbors(adata_int, use_rep='X_emb')
sc.tl.umap(adata_int)
sc.pl.umap(adata_int, color=cell_type, frameon=False)
sc.pl.umap(adata_int, color=batch, frameon=False)

In [ ]:
obs_names = sc.read_h5ad(data_path).obs_names.to_list()
adata_int.obs_names = pd.Series(adata_int.obs_names).apply(lambda x:x.rsplit('-',1)[0])
adata_int = adata_int[obs_names,]

In [ ]:
pd.DataFrame(adata_int.obsm['X_emb']).to_csv('scDML-'+dataset+'.csv',header = False,index = False)

In [ ]:
import sys
sys.path.append('/home/krushna/Documents/Data_integration/')
import scIB

In [ ]:
adata = sc.read_h5ad(data_path)
adata_int = adata.copy()
adata_int.obsm['X_emb'] = pd.read_csv('scDML-'+dataset+'.csv',index_col=None,header = None).values

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

sc.pp.neighbors(adata_int, use_rep='X_emb')
sc.tl.umap(adata_int)
sc.pl.umap(adata_int, color=cell_type, frameon=False)
sc.pl.umap(adata_int, color=batch, frameon=False)

In [ ]:
import scIB 
#Trajectory is asking precomputed sudo time point
results,ilisi_all,clisi_all,kbet_all  =      scIB.metrics.metrics(
        adata,
        adata_int,
        batch_key = batch,
        label_key = cell_type,
        hvg_score_=False,
        cluster_key='cluster',
        cluster_nmi=None,
        ari_=True,
        nmi_=True,
        nmi_method='arithmetic',
        nmi_dir=None,
        silhouette_=True,
        embed= 'X_emb',
        si_metric='euclidean',
        pcr_=True,
        cell_cycle_=False,
        organism='mouse',
        isolated_labels_=True,  # backwards compatibility
        isolated_labels_f1_=True,
        isolated_labels_asw_=True,
        n_isolated=None,
        graph_conn_=True,
        kBET_=True,
        kBET_sub=0.5,
        lisi_graph_=True,
        lisi_raw=True,
        trajectory_=False,
        type_=None,
        verbose=False,
)
results

In [ ]:
import numpy as np
np.savetxt('./kbet_lisi/'+dataset+"_ilisi.csv", ilisi_all, delimiter=",")
np.savetxt('./kbet_lisi/'+dataset+"_clisi.csv", clisi_all, delimiter=",")
np.savetxt('./kbet_lisi/'+dataset+"_kbet_all.csv",np.concatenate([np.array(val).reshape(1,-1) for val in kbet_all],axis = 0), delimiter=',')